# Run Experiments

## Imports

In [1]:
# Disables generation of pycache file
import sys
sys.dont_write_bytecode = True

# Imports libraries
import pandas as pd

# Imports made functions
from solvers import *
from utility import *

## Running Experiments

In [2]:
result_columns_basenames = ["||H||_1" , "||H||_0", "rank(H)", "||AHA - A||_F", "||HAH - H||_F",
                "||(AH)^T - AH||_F", "||A^TAH - AT||_F"]

problems = ["1_norm_P1", "1_norm_PLS", "1_norm_P1_PLS", "1_norm_P1_P3"]

result_column_names = []
for problem in problems:
    for basename in result_columns_basenames:
        result_column_names.append(f"{problem}_{basename}")

column_names = ["m", "n", "r"] + result_column_names

m_values = [10*i for i in range(5, 11)]

In [3]:
solvers = {
    "1_norm_P1": problem_1_norm_P1_solver,
    "1_norm_PLS": problem_1_norm_PLS_solver,
    "1_norm_P1_PLS": problem_1_norm_P1_PLS_solver,
    "1_norm_P1_P3": problem_1_norm_P1_P3_solver
}

is_viable_checks = {
    "1_norm_P1": problem_1_norm_P1_viable_solution,
    "1_norm_PLS": problem_1_norm_PLS_viable_solution,
    "1_norm_P1_PLS": problem_1_norm_P1_PLS_viable_solution,
    "1_norm_P1_P3": problem_1_norm_P1_P3_viable_solution
}

In [4]:
try:
    df = pd.read_csv("results.csv")
except:
    # Creates an empty dataframe with the specified column names
    df = pd.DataFrame(columns=column_names)

    # Saves dataframe as a csv file
    df.to_csv('results.csv', index=False)

In [5]:
df = pd.read_csv("results.csv")

In [6]:
for m in m_values:
    A = generate_random_rank_r_matrix(m=m)
    n = int(0.5 * m)
    r = int(np.floor(0.25 * m))
    instance_results = {
        "m": m,
        "n": n,
        "r": r
    }

    for i in range(len(problems)):
        problem = problems[i]
        solver = solvers[problem]
        is_viable_check = is_viable_checks[problem]
        H_star = solver(A=A)
        if (not is_viable_check(A=A, H=H_star, m=m, n=n)):
            print(f"m: {m}")
            print(f"problem: {problem} did not find a viable solution")
            break
        problem_results = calculate_problem_results(A=A, H=H_star, problem=problem)
        for key, value in problem_results.items():
            instance_results[key] = value

    df.loc[len(df)] = instance_results
    df.to_csv('results.csv', index=False)

In [10]:
df.to_excel('results.xlsx', sheet_name='1_norm_experiments', index=False)